In this file, we check correlation between survey values for each party and article number for the wars with most media presens. 
To check whether the correlation is statistically significant, we do a permutation test for each correlation value and obtain a p-value.

In [164]:
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import datetime
import numpy as np
import math

read_path = r'C:\Users\XeniaRieger\OneDrive - bwedu\Dokumente\Studium Master\1. Semester\Data Literacy\Data-Literacy\Data\joined_survey_article_standardized_median.csv'
df = pd.read_csv(read_path, delimiter=';', header=0)
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")
df["SPD"] = pd.to_numeric(df["SPD"], errors='coerce')
df["Union"] = pd.to_numeric(df["Union"], errors='coerce')
df["Grüne"] = pd.to_numeric(df["Grüne"], errors='coerce')
df["FDP"] = pd.to_numeric(df["FDP"], errors='coerce')
df["AfD"] = pd.to_numeric(df["AfD"], errors='coerce')
df["Linke"] = pd.to_numeric(df["Linke"], errors='coerce')

First we define the timeframes to test. In our opinion it does not make sense to calculate correlation for all years: due to different opinions on each war a party might have positive or negative correlations for different wars.
The wars with the most media presens are defined by the highest article numbers:
**2003: january, february, march, april -> Irakkrieg**
2008: August -> Russland und Georgien
2009: januray -> Operation Gegossenes Blei, Israel und der Hamas im Gazastreifen.
**2022 and 2023: all year -> verschiedene Konflikte, Russland und Ukraine**
We decide not to calculate correlation for 2008 and 2009, as both wars are media present only for one to two surveys. For war in Irak and Ukraine we look at the correlation of each parties values with the article number. For Irak war we add half a year before and after media presence. For Ukraine war we start at the first media presenz and calcuate correlation until our last values, as the conflict is ongoing.

In [56]:
this_df = df[(df['date'].dt.year >= 2002) & (df['date'].dt.year <= 2004)]
high_values = this_df[this_df['standardized_war_articles'] >= 0.2]
first = high_values['date'].iloc[-1]
last = high_values['date'].iloc[0]
new_first = first - datetime.timedelta(days=6*30)
new_last = last + datetime.timedelta(days=6*30)
war_df_irak = df[(df['date'] >= new_first) & (df['date'] <= new_last)]

In [57]:
correlation_matrix_irak = war_df_irak.corr()
correlation_matrix_irak['standardized_war_articles']

Unnamed: 0                   0.004723
SPD                         -0.293295
Union                        0.368109
Grüne                        0.388835
FDP                         -0.373680
AfD                               NaN
Linke                       -0.551424
standardized_war_articles    1.000000
Name: standardized_war_articles, dtype: float64

In [58]:
this_df = df[(df['date'].dt.year >= 2022) & (df['date'].dt.year <= 2023)]
high_values = this_df[this_df['standardized_war_articles'] >= 0.2]
first = high_values['date'].iloc[-1]
new_first = first - datetime.timedelta(days=6*30)
war_df_ukraine = df[(df['date'] >= new_first)]

In [69]:
correlation_matrix_ukraine = war_df_ukraine.corr()
correlation_matrix_ukraine['standardized_war_articles']

-0.13186951807660338

In [68]:
df_res = pd.DataFrame({"Irak": correlation_matrix_irak['standardized_war_articles'],
                       "Ukraine": correlation_matrix_ukraine['standardized_war_articles']})
df_res = df_res.drop(["Unnamed: 0", "standardized_war_articles"])
df_res

,Irak,Ukraine
SPD,-0.293295,-0.131870
Union,0.368109,0.359961
Grüne,0.388835,0.129753
FDP,-0.373680,-0.393386
AfD,NaN,0.039217
Linke,-0.551424,-0.365879


It seems like there are similar correlations for each party for both wars. To check whether the correlation is statistically significant, we do a permutation test for each correlation value and obtain a p-value. For that we permute the article values for the investigated timeframe.

In [132]:
def permutation_test(df_test, corr_matrix, party):
    B = 10000
    corr_values = []
    extrem_values = 0
    for x in range(B):
        df_permuted = df_test.copy()
        # permute exising values
        df_permuted["standardized_war_articles"] = np.random.permutation(df_permuted["standardized_war_articles"])
        corr_permuted = df_permuted.corr()["standardized_war_articles"][party]
        corr_values.append(corr_permuted)
        true_value = corr_matrix["standardized_war_articles"][party]
        if abs(corr_permuted) > abs(true_value):
            extrem_values += 1

    p_value = extrem_values / B        
    #fig, axs = plt.subplots()
    #axs.hist(corr_values)
    #axs.axvline(corr_matrix["standardized_war_articles"][party], alpha=1.0, color="red")
    return p_value

In [138]:
p_values_irak = [permutation_test(war_df_irak, correlation_matrix_irak, "SPD"),
                 permutation_test(war_df_irak, correlation_matrix_irak, "Union"),
                 permutation_test(war_df_irak, correlation_matrix_irak, "Grüne"),
                 permutation_test(war_df_irak, correlation_matrix_irak, "FDP"),
                 float('nan'),
                 permutation_test(war_df_irak, correlation_matrix_irak, "Linke")]

p_values_ukraine = [permutation_test(war_df_ukraine, correlation_matrix_ukraine, "SPD"),
                    permutation_test(war_df_ukraine, correlation_matrix_ukraine, "Union"),
                    permutation_test(war_df_ukraine, correlation_matrix_ukraine, "Grüne"),
                    permutation_test(war_df_ukraine, correlation_matrix_ukraine, "FDP"),
                    permutation_test(war_df_ukraine, correlation_matrix_ukraine, "AfD"),
                    permutation_test(war_df_ukraine, correlation_matrix_ukraine, "Linke")]

df_pvalues = pd.DataFrame({"Irak": p_values_irak, "Ukraine": p_values_ukraine}, index=["SPD", "Union", "Grüne", "FDP", "AfD", "Linke"])
df_pvalues

In [149]:
df_pvalues

,Irak,Ukraine
SPD,0.0214,0.3439
Union,0.0022,0.0073
Grüne,0.0026,0.3496
FDP,0.0029,0.0025
AfD,NaN,0.7800
Linke,0.0000,0.0066


In [166]:
res_significant = pd.DataFrame(index=["SPD", "Union", "Grüne", "FDP", "AfD", "Linke"], columns=["Irak", "Ukraine"])
for i in ["Irak", "Ukraine"]:
    for j in ["SPD", "Union", "Grüne", "FDP", "AfD", "Linke"]:
        if math.isnan(df_pvalues[i][j]):
            continue
        if df_pvalues[i][j] <= 0.05:
            res_significant[i][j] = str(round(df_res[i][j], 2)) + "*"
        else:
            res_significant[i][j] = str(round(df_res[i][j], 2))
res_significant

,Irak,Ukraine
SPD,-0.29*,-0.13
Union,0.37*,0.36*
Grüne,0.39*,0.13
FDP,-0.37*,-0.39*
AfD,NaN,0.04
Linke,-0.55*,-0.37*
